# Variant Calling on SARS-CoV-2 NGS Data

This notebook summarises step by step a miminal workflow, to use raw NGS data, generate a list of detected variants in each sample, and assign a pangolin lineage to each one.

## Initial set-up

We begin by organising our working environment: first, let's create a new folder where we are going to save the data in our home directory.

In [ ]:
cd ~
mkdir -p variantlab
cd variantlab 

The genome reference file is already available in the DATA folder, but we will copy in our exercise folder and rename it for convenience, as it follows:

In [ ]:
cp /home/student/DATA/variant_calling_data/refs/refseq_NC_045512_covid19_wuhan.fasta covid_reference.fasta

As we discuss in class, the genome needs to be indexed in order to allow the mapping algorithm to work. We have therefore to create the indexes first.

Since these are made by several different files, we want to keep our folders organised and we will create a specific folder to store them.

In [ ]:
mkdir -p bowtie2

Now we can use bowtie, in order to build the indexes as follows:

In [ ]:
bowtie2-build \
--threads 2 \
covid_reference.fasta \
bowtie2/covid_reference

## Aligning the NGS reads to SARS-CoV-2 genome

Now we are ready to run the alignment for each sample, i.e. each pair of forward and reverse reads.

As usual, we want to keep things tidy and we are going to create a folder where we will store the alignment files.

In [ ]:
cd ~/variantlab
mkdir -p alignment
cd alignment

Now we are ready to run the mapping software first on sample one, as follows:

In [ ]:
bowtie2 \
    -x ~/variantlab/bowtie2/covid_reference \
    -1 /home/student/DATA/variant_calling_data/fastq/sample1_R1.fastq.gz \
    -2 /home/student/DATA/variant_calling_data/fastq/sample1_R2.fastq.gz \
    --threads 2 \
    2> sample1_bowtie2.log \
    | samtools view -@ 2 -bhS -o sample1.bam -


Then on sample 2, as follows:

In [ ]:
bowtie2 \
    -x ~/variantlab/bowtie2/covid_reference \
    -1 /home/student/DATA/variant_calling_data/fastq/sample2_R1.fastq.gz \
    -2 /home/student/DATA/variant_calling_data/fastq/sample2_R2.fastq.gz \
    --threads 2 \
    2> sample1_bowtie2.log \
    | samtools view -@ 2 -bhS -o sample2.bam -

The BAM files we have just generated need to be indexed as well: as for the reference genome, an index will allow quick access and navigation through the data. 
In order to generate an index, we need first to sort the file.
You will notice that the command also include the generation of an index at the end, which is written in a recently developed format.

In [ ]:
samtools sort -@ 2 --write-index -o sample1_sorted.bam sample1.bam
samtools sort -@ 2 --write-index -o sample2_sorted.bam sample2.bam

However, some of the downstream tools we are going to use still prefer a more traditional index file, which we will generate with the following code:

In [ ]:
samtools index sample1_sorted.bam
samtools index sample2_sorted.bam

## Variant Calling

As usual, we first create a folder where to store the variants and the associated files:

In [ ]:
cd ~/variantlab
mkdir -p calls
cd calls

We assign the file location of the two previously generated bam files to a bash variable for convenience:

In [ ]:
bam1=/home/student/variantlab/alignment/sample1_sorted.bam
bam2=/home/student/variantlab/alignment/sample2_sorted.bam

And we are doing the same with the additional files we need:

- a primer file: this is used to trim the data, removing regions where variants might be incorrectly called
- an annotation file, i.e. a descrition of the SARS-CoV-2 genome which is used to understand where variants are 
- the reference genome file

We assign these to variables as follows:

In [ ]:
primers=/home/student/DATA/variant_calling_data/refs/nCoV-2019.primer.bed
gff=/home/student/DATA/variant_calling_data/refs/GCF_009858895.2_ASM985889v3_genomic.gff
fasta=/home/student/variantlab/covid_reference.fasta

Now we are ready to trim the primers from both bam files:

In [ ]:
ivar trim \
  -i $bam1 \
  -b $primers \
  -e -p "sample1_primer_trimmed"


ivar trim \
  -i $bam2 \
  -b $primers \
  -e -p "sample2_primer_trimmed"

And like we did before, we need to sort and index the resulting trimmed bam files:

In [ ]:
samtools sort -@ 2 --write-index -o sample1_primer_sorted.bam sample1_primer_trimmed.bam
samtools sort -@ 2 --write-index -o sample2_primer_sorted.bam sample2_primer_trimmed.bam

The files are now ready to be used for variant calling, first on the first sample:

In [ ]:
samtools mpileup \
  -aa -A -d 0 -B -Q 0 \
  --reference $fasta \
  sample1_primer_sorted.bam \
  | ivar variants -p sample1_variants \
  -t 0.01 \
  -m 5 \
  -r $fasta \
  -g $gff

Then on the second sample:

In [ ]:
samtools mpileup \
  -aa -A -d 0 -B -Q 0 \
  --reference $fasta \
  sample2_primer_sorted.bam \
  | ivar variants -p sample2_variants \
  -t 0.01 \
  -m 5 \
  -r $fasta \
  -g $gff

## Consensus Genome

We first create a folder where to store the files we need for the consensus

In [ ]:
cd ~/variantlab
mkdir -p consensus
cd consensus

And we use ivar to create a consensus genome for the first sample:

In [ ]:
samtools mpileup -aa -A -d 0 -Q 0 \
  /home/student/variantlab/calls/sample1_primer_sorted.bam \
  | ivar consensus \
  -t 0.1 \
  -m 5 \
  -p sample1_consensus

then for the second sample

In [ ]:
samtools mpileup -aa -A -d 0 -Q 0 \
  /home/student/variantlab/calls/sample2_primer_sorted.bam \
  | ivar consensus \
  -t 0.1 \
  -m 5 \
  -p sample2_consensus

Now we have 2 fasta files representing the genomes of the 2 samples with the variants that we have discovered in them.

## Lineage Assignment

In order to classify them both at the same time, we can collate them in the same multi-fasta file, as follows:

In [ ]:
cd ~/variantlab/consensus

cat sample1_consensus.fa sample2_consensus.fa >samples_consensus.fa

We need to create a temp folder, to allow the lineage assignment software to work properly

In [ ]:
mkdir -p tmp

And we can now use the Pangolin tool to run the classification algorithm

In [ ]:
pangolin -t 2 \
--tempdir ~/variantlab/consensus/tmp \
samples_consensus.fa

We now have a CSV file, reporting the lineage assigned to each sample. 
let's inspect the results.

Then, for the last step we have to move to the RStudio environment.